# PROJET 3 : Etude de santé publique 

### Partie 2 : les codes python m'ayant permis d'enregistrer les données dans la base de données et de repondre aux questions de 15 à 18. 

### Entrez vos données dans une base de données relationnelle

In [7]:
import pandas as pd
import numpy as np
import sqlite3 as sql

### Question 15 : Proposez une clé primaire pertinente pour cette table.
#### Préparation, nettoyage et création de la table population

In [8]:
# Lecture de la table
population=pd.read_csv('fr_population.csv', sep=',', header=0)
#Suppression de Chine car doublons avec Chine continentale
population= population[population['Code zone'] != 351]
# Projection des colonnes du dataframe utiles pour l'étude et multiplication de "Valeur" par mille "Unité"
population=population[['Zone','Code zone','Année','Valeur']]
population['Valeur'] *=1000
# renommer la colonne Valeur par Population
population=population.rename({'Zone':'pays','Code zone':'code_pays','Année':'annee','Valeur':'population'}, axis=1)
population

,pays,code_pays,annee,population
0,Afghanistan,2,2013,30552000
1,Afrique du Sud,202,2013,52776000
2,Albanie,3,2013,3173000
3,Algérie,4,2013,39208000
4,Allemagne,79,2013,82727000
...,...,...,...,...
170,Venezuela (République bolivarienne du),236,2013,30405000
171,Viet Nam,237,2013,91680000
172,Yémen,249,2013,24407000
173,Zambie,251,2013,14539000


###### la clé primaire pour cette table est : Code pays, annee

In [9]:
# création table csv
population.to_csv('population.csv',index=False)

In [10]:
# création et connection à la base de donnée FAO
cnn=sql.connect('FAO.db')
# upload du dataframe population dans la base de données FAO 
population.to_sql('population',cnn, if_exists='replace',index=False)

### Question 16 : Proposez une clé primaire pertinente pour cette table
###### Une table appelée dispo_alim contenant pour chaque pays et pour chaque produit en 2013, les informations suivantes:
###### la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
###### disponibilité alimentaire en tonnes
###### disponibilité alimentaire en Kcal/personne/jour
###### disponibilité alimentaire de protéines en g/personne/jour
###### disponibilité alimentaire de matières grasses en g/personne/jour
###### Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, origine, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .

#### Préparation, nettoyage et création de la table dispo_alim

In [11]:
# Lecture des table animaux et vegetaux et creation d'une colonne origine pour la tracabilité 
animaux = pd.read_csv('fr_animaux.csv', sep=',', header=0)
animaux['origine'] = 'animale'
# Lecture des table animaux et vagetaux et creation d'une colonne origine pour la tracabilité 
vegetaux = pd.read_csv('fr_vegetaux.csv', sep=',', header=0)
vegetaux['origine'] = 'vegetale'
# Suppression de Chine car doublons avec Chine continentale
animaux=animaux[animaux['Code zone'] != 351]
vegetaux=vegetaux[vegetaux['Code zone'] != 351]
# Concatenation du dataframe cerealesVegetaux et animaux car structures identiques
vegetauxAnimaux=pd.concat([vegetaux,animaux],)
# renommer les colonnes
vegetauxAnimaux=vegetauxAnimaux.rename({'Code zone' : 'code_pays','Zone' : 'pays','Code Élément': 'code_element','Élément':'element','Code Produit':'code_produit','Produit': 'produit','Année': 'annee','Unité':'unite', 'Valeur':'valeur'}, axis=1)
# Jointure de population
vegetauxAnimauxPopulation=pd.merge(vegetauxAnimaux, population[['pays','population']], on='pays', how='inner')
# mise en perspective de la table dispo_alim
vegetauxAnimauxPopulation
vegetauxAnimauxPopulation=vegetauxAnimauxPopulation.pivot_table(index=['code_pays', 'pays', 'code_produit', 'produit', 'annee','origine','population'],columns=['element'],values=['valeur'],aggfunc=sum)
vegetauxAnimauxPopulation.columns=['Aliments pour animaux','Autres utilisations (non alimentaire)', 'Disponibilité alimentaire (Kcal/personne/jour)','Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité de matière grasse en quantité (g/personne/jour)', 'Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 'Production', 'Semences', 'Traitement', 'Variation de stock']
# Renommer les colonnes pour la syntaxe et pour mettre en adéquation les unités avec les elements
vegetauxAnimauxPopulation=vegetauxAnimauxPopulation.rename(columns={'Disponibilité intérieure':'dispo_int','Exportations - Quantité':'exportKtonne','Aliments pour animaux':'alim_ani','Nourriture':'nourriture','Disponibilité alimentaire (Kcal/personne/jour)':'dispo_alim_kcal_p_j','Disponibilité alimentaire en quantité (kg/personne/an)':'dispo_alim_Kg_pers_an','Importations - Quantité':'importaKtonne','Autres utilisations (non alimentaire)':'autres_utilisations','Traitement':'transfo','Production':'productionKtonne','Disponibilité de matière grasse en quantité (g/personne/jour)': 'dispo_mat_gr','Disponibilité de protéines en quantité (g/personne/jour)':'dispo_prot','Semences':'semences','Variation de stock':'varStockKtonne','Pertes':'pertes'})
vegetauxAnimauxPopulation=vegetauxAnimauxPopulation.reset_index()
# projection des colonnes pour la table dispo_alim
dispo_alim=vegetauxAnimauxPopulation[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'origine', 'dispo_alim_Kg_pers_an', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr','population']]
# calcul de la disponibilité alimentaire en tonnes
dispo_alim=dispo_alim.dropna()
dispo_alim['dispo_alim_tonnes']=(dispo_alim['dispo_alim_Kg_pers_an']*dispo_alim['population']/1000)
dispo_alim=dispo_alim[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'origine', 'dispo_alim_tonnes', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr']]
dispo_alim=dispo_alim.dropna(subset=['dispo_alim_tonnes', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr'],how='all')
dispo_alim

,pays,code_pays,annee,produit,code_produit,origine,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr
0,Arménie,1,2013,Blé,2511,vegetale,388796.20,1024.0,30.52,3.60
1,Arménie,1,2013,Orge,2513,vegetale,0.00,0.0,0.00,0.00
3,Arménie,1,2013,Seigle,2515,vegetale,357.24,1.0,0.02,0.00
4,Arménie,1,2013,Avoine,2516,vegetale,1101.49,2.0,0.09,0.03
7,Arménie,1,2013,"Céréales, Autres",2520,vegetale,5834.92,14.0,0.39,0.04
...,...,...,...,...,...,...,...,...,...,...
15599,Soudan,276,2013,Perciform,2762,animale,0.00,0.0,0.00,0.00
15600,Soudan,276,2013,Poissons Pelagiques,2763,animale,0.00,0.0,0.00,0.00
15601,Soudan,276,2013,"Poissons Marins, Autres",2764,animale,4935.32,0.0,0.04,0.01
15602,Soudan,276,2013,Riz (Eq Blanchi),2805,vegetale,48214.28,14.0,0.28,0.05


###### la clé primaire pour cette table est : code pays,code produit,annee

In [7]:
# creation table csv
dispo_alim.to_csv('dispo_alim.csv',index=False)

In [8]:
# upload du dataframe dispo_alim dans la base de données FAO 
dispo_alim.to_sql('dispo_alim',cnn, if_exists='replace',index=False)

### Question 17 : Proposez une clé primaire pertinente pour cette table.
######  Une table appelée equilibre_prod contenant pour chaque pays et pour chaque produit en 2013, les quantités suivantes :
######  disponibilité intérieure
###### aliments pour animaux
###### semences
###### pertes
###### transformés
###### nourriture
###### autres utilisations
###### Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations

#### Préparation,nettoyage et création de la table equilibre_prod

In [9]:
# projection des colonnes pour equilibre_prod
equilibre_prod=vegetauxAnimauxPopulation[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'dispo_int', 'alim_ani', 'semences', 'pertes', 'transfo', 'nourriture', 'autres_utilisations' ]]
equilibre_prod=equilibre_prod.dropna()
equilibre_prod

,pays,code_pays,annee,produit,code_produit,dispo_int,alim_ani,semences,pertes,transfo,nourriture,autres_utilisations
0,Arménie,1,2013,Blé,2511,554.0,93.0,30.0,32.0,10.0,389.0,0.0
1,Arménie,1,2013,Orge,2513,198.0,137.0,14.0,15.0,7.0,0.0,26.0
8,Arménie,1,2013,Pommes de Terre,2531,672.0,225.0,123.0,96.0,78.0,134.0,16.0
151,Albanie,3,2013,Orge,2513,19.0,7.0,0.0,1.0,9.0,1.0,1.0
152,Albanie,3,2013,Maïs,2514,420.0,361.0,3.0,44.0,3.0,6.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15341,Serbie,272,2013,Orge,2513,346.0,205.0,18.0,8.0,104.0,10.0,1.0
15342,Serbie,272,2013,Maïs,2514,4996.0,4309.0,30.0,294.0,144.0,219.0,1.0
15359,Serbie,272,2013,Soja,2555,272.0,11.0,6.0,20.0,230.0,0.0,4.0
15433,Monténégro,273,2013,Orge,2513,7.0,0.0,0.0,0.0,3.0,0.0,4.0


###### la clé primaire pour cette table est :code pays, code produit,annee

In [10]:
# creation table csv
equilibre_prod.to_csv('equilibre_prod.csv',index=False)

In [11]:
# upload du dataframe equilibre_prod dans la base de données FAO 
equilibre_prod.to_sql('equilibre_prod',cnn, if_exists='replace',index=False)

### Question 18 : Vous vous en doutez... proposez encore une fois une clé primaire pertinente pour cette table !
###### Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays en 2013. Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.
#### Préparation,nettoyage et création de la table sous_nutrition

In [12]:
# lecture de la table 
sous_alimentation= pd.read_csv('fr_sousalimentation.csv', sep=',', header=0)
# création et projection des colonnes dont on à besoin à savoir: pays, code_pays, annee, population et multiplication de "Valeur" par mille "Unité"
sous_nutrition=sous_alimentation[['Zone', 'Code zone','Année', 'Valeur']]
#suppression de <0.1
sous_nutrition=sous_nutrition[sous_nutrition['Valeur'] != '<0.1']
#suppression NaN
sous_nutrition=sous_nutrition.dropna()
# transformation du type de valeur
sous_nutrition['Valeur']=sous_nutrition.Valeur.astype(float)
# multiplication pour la coherence avec les unités
sous_nutrition['Valeur'] *=1e6
#Suppression de Chine car doublons avec Chine continentale
sous_nutrition=sous_nutrition[sous_nutrition['Code zone']!= 351]
# renommer les colonnes
sous_nutrition=sous_nutrition.rename({'Zone' : 'pays','Code zone' : 'code_pays','Année': 'annee', 'Valeur':'nb_personnes'}, axis=1)
# on change les valeurs de "annee" pour qu'elles puissent correspondre aux autres dataframes
sous_nutrition.replace('2011-2013', '2012', inplace=True)
sous_nutrition.replace('2012-2014', '2013',  inplace=True)
sous_nutrition.replace('2013-2015', '2014',  inplace=True)
sous_nutrition.replace('2014-2016', '2015',  inplace=True)
sous_nutrition.replace('2015-2017', '2016',  inplace=True)
sous_nutrition.replace('2016-2018', '2017',  inplace=True)
sous_nutrition

,pays,code_pays,annee,nb_personnes
0,Afghanistan,2,2013,7900000.0
1,Afghanistan,2,2014,8800000.0
2,Afghanistan,2,2015,9600000.0
3,Afghanistan,2,2016,10200000.0
4,Afghanistan,2,2017,10600000.0
...,...,...,...,...
1015,Zimbabwe,181,2013,6600000.0
1016,Zimbabwe,181,2014,7200000.0
1017,Zimbabwe,181,2015,7800000.0
1018,Zimbabwe,181,2016,8200000.0


###### la clé primaire pour cette table est : code pays, annee

In [13]:
# creation table csv
sous_nutrition.to_csv('sous_nutrition.csv',index=False)

In [14]:
# upload du dataframe sous_nutrition dans la base de données FAO 
sous_nutrition.to_sql('sous_nutrition',cnn, if_exists='replace',index=False)